<a href="https://colab.research.google.com/github/aboelela924/tensorFlowTutorials/blob/master/TextClassficationWithHubTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##links 
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [ ]:
!pip install tfds-nightly
!pip install tensorflow-hub

In [3]:
import numpy as np 
import tensorflow as tf 
import tensorflow_datasets as tfds
import tensorflow_hub as tfhub

In [11]:
print(f"Tensorflow version: {tf.__version__}")
print(f"Eager mode: {tf.executing_eagerly()}")
print(f"Hub version: {tfhub.__version__}")
print(f"Do we have GPU? {'Yes' if tf.config.experimental.list_physical_devices('GPU') else 'No'}")

Tensorflow version: 2.2.0
Eager mode: True
Hub version: 0.8.0
Do we have GPU? Yes


In [67]:
BATCH_SIZE = 128

In [76]:
ds_train, train_info = tfds.load("imdb_reviews", split="train", data_dir="/content/data", 
               batch_size=BATCH_SIZE, shuffle_files=True, as_supervised=True, 
               with_info=True)

ds_test, test_info = tfds.load("imdb_reviews", split="test", data_dir="/content/data", 
               batch_size=BATCH_SIZE, shuffle_files=True, as_supervised=True, 
               with_info=True)

ds_train = ds_train.repeat()

In [69]:
num_of_train_samples = train_info.splits["train"].num_examples
num_of_test_samples = train_info.splits["test"].num_examples
steps_per_epoch =  num_of_train_samples//num_of_train_samples

In [61]:
classes_names = ["Negative", "Positive"]

In [70]:
sample_text, sample_labels = next(iter(ds_train))
print(sample_text[0])
print(classes_names[sample_labels[0]])

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
Negative


In [ ]:
print(train_info)

In [71]:
embedding_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_embedding_layer = tfhub.KerasLayer(embedding_model, trainable=True,
                                       output_shape=[20], input_shape=[],
                                       dtype=tf.string)


In [93]:
model = tf.keras.Sequential([
                             hub_embedding_layer,
                             tf.keras.layers.Reshape((20, 1)),
                             tf.keras.layers.LSTM(20),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(12, activation="relu"),
                             tf.keras.layers.Dense(1, activation="sigmoid")])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
reshape_4 (Reshape)          (None, 20, 1)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20)                1760      
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 12)                252       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 13        
Total params: 402,045
Trainable params: 402,045
Non-trainable params: 0
________________________________________________

In [94]:
model.compile(optimizer="adam",
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [95]:
model.fit_generator(ds_train, steps_per_epoch=steps_per_epoch, epochs=200, 
                    verbose=2, validation_data=ds_test, validation_steps=30)

Epoch 1/200
1/1 - 1s - loss: 0.7162 - accuracy: 0.4922 - val_loss: 0.7219 - val_accuracy: 0.4961
Epoch 2/200
1/1 - 0s - loss: 0.7342 - accuracy: 0.4688 - val_loss: 0.7217 - val_accuracy: 0.4969
Epoch 3/200
1/1 - 0s - loss: 0.7077 - accuracy: 0.4766 - val_loss: 0.7216 - val_accuracy: 0.4971
Epoch 4/200
1/1 - 0s - loss: 0.7349 - accuracy: 0.4766 - val_loss: 0.7214 - val_accuracy: 0.4971
Epoch 5/200
1/1 - 0s - loss: 0.6773 - accuracy: 0.4766 - val_loss: 0.7212 - val_accuracy: 0.4971
Epoch 6/200
1/1 - 0s - loss: 0.7137 - accuracy: 0.5156 - val_loss: 0.7210 - val_accuracy: 0.4971
Epoch 7/200
1/1 - 0s - loss: 0.7312 - accuracy: 0.5078 - val_loss: 0.7208 - val_accuracy: 0.4971
Epoch 8/200
1/1 - 0s - loss: 0.7213 - accuracy: 0.5391 - val_loss: 0.7206 - val_accuracy: 0.4971
Epoch 9/200
1/1 - 0s - loss: 0.7367 - accuracy: 0.5234 - val_loss: 0.7204 - val_accuracy: 0.4971
Epoch 10/200
1/1 - 0s - loss: 0.7352 - accuracy: 0.5391 - val_loss: 0.7201 - val_accuracy: 0.4971
Epoch 11/200
1/1 - 0s - loss:

In [96]:
test_negative_review = ["This movie is terrible. I hated it."]
test_positive_review = ["This movie is marvelous. I loved it."]
prediction = model.predict(test_negative_review)
print(prediction)
prediction = model.predict(test_positive_review)
print(prediction)

[[0.00732741]]
[[0.973032]]
